# Ollama + OpenAI + Python

## 1. Ange modellnamnet

Om du har hämtat en annan modell än "phi3:mini", ändra värdet i cellen nedan. Den variabeln kommer att användas i kod genom hela notebooken.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Konfigurera Open AI-klienten

Vanligtvis används OpenAI-klienten med OpenAI.com eller Azure OpenAI för att interagera med stora språkmodeller.  
Men den kan också användas med Ollama, eftersom Ollama erbjuder en OpenAI-kompatibel endpoint på "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Generera ett chatt-svar

Nu kan vi använda OpenAI SDK för att skapa ett svar på en konversation. Den här förfrågan bör generera en haiku om katter:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Promptteknik

Det första meddelandet som skickas till språkmodellen kallas "systemmeddelande" eller "systemprompt", och det sätter de övergripande instruktionerna för modellen.  
Du kan skapa din egen anpassade systemprompt för att styra en språkmodell att generera output på ett annat sätt.  
Ändra `SYSTEM_MESSAGE` nedan för att svara som din favoritkaraktär från en film eller TV-serie, eller hämta inspiration för andra systemprompter från [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

När du har anpassat systemmeddelandet, ge den första användarfrågan i `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Exempel med få skott

Ett annat sätt att styra en språkmodell är att ge "få skott", en sekvens av exempel på frågor/svar som visar hur den ska svara.

Exemplet nedan försöker få en språkmodell att agera som en lärarassistent genom att ge några exempel på frågor och svar som en lärarassistent kan ge, och sedan ställa en fråga som en student skulle kunna fråga.

Prova det först, och ändra sedan `SYSTEM_MESSAGE`, `EXAMPLES` och `USER_MESSAGE` för ett nytt scenario.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Retrieval Augmented Generation

RAG (Retrieval Augmented Generation) är en teknik för att få en språkmodell att svara korrekt på frågor inom ett specifikt område, genom att först hämta relevant information från en kunskapskälla och sedan generera ett svar baserat på den informationen.

Vi har tillhandahållit en lokal CSV-fil med data om hybridbilar. Koden nedan läser CSV-filen, söker efter matchningar till användarens fråga och genererar sedan ett svar baserat på den hittade informationen. Observera att detta kommer att ta längre tid än de tidigare exemplen, eftersom mer data skickas till modellen. Om du märker att svaret fortfarande inte är baserat på datan, kan du prova systemteknik eller testa andra modeller. Generellt sett är RAG mer effektivt med antingen större modeller eller med finjusterade versioner av SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör det noteras att automatiserade översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som kan uppstå vid användning av denna översättning.
